# Métodos iterativos
- Óscar Alvarado Morán

In [1]:
using LinearAlgebra
using Plots
using SparseArrays

#### Diferencias finitas

In [2]:
function FD_Matrix(N, nodos, h, tipo = :F)
    A = zeros(nodos, nodos)
    # Forward
    if tipo == :F 
        v = [((binomial(N, a))*(-1)^a)/(h^N) for a = N:-1:0]
        for i in 0:N
            A += diagm(i => [v[i+1] for j = 1:(nodos-i)])
        end
        return A
    # Backward
    elseif tipo == :B 
        v = [((binomial(N, a))*(-1)^a)/(h^N) for a = 0:N]
        for i in 0:N
            A += diagm(-i => [v[i+1] for j = 1:(nodos-i)])
        end
        return A
    # Central
    elseif tipo == :C
        v = [((binomial(N, a))*(-1)^a)/(h^N) for a = 0:N]
        if (N + 1) % 2 == 0
            for i in (N+1)/2:-1:1
                A += diagm(convert(UInt8, i) => [v[convert(UInt8, ((N+1)/2)-i+1)] for j = 1:(nodos-i)])
                A += diagm(-1*convert(UInt8, i) => [v[convert(UInt8, end - (((N+1)/2)-i))] for j = 1:(nodos-i)])
            end
        else
            for i in 0:N
                A += diagm(i-convert(UInt8, (N/2)) => [v[i+1] for j = 1:(nodos-abs(i-convert(UInt8, (N/2))))])
            end
        end
        return A
    else
        print("No")
    end
end

FD_Matrix (generic function with 2 methods)

In [3]:
function Ejercicio1_1(n, inicio, final, b, condiciones = false)
    h = (final-inicio)/(n-1) # Tamaño del intervalo sobre el eje x
    x = inicio:h:final
    
    non_zeros = findall(!iszero, b)
    inverse = [1/i for i in x]
    
    for i in non_zeros
        inverse[i] = 0
    end
    A = FD_Matrix(2, n, h, :F) + Diagonal(inverse) * FD_Matrix(1, n, h, :F) - Diagonal(inverse .^ 2 )
    
    if condiciones == true
        A[non_zeros, :] = zeros(length(non_zeros), size(A)[1])

        for j in non_zeros
            A[j, j] = 1
        end
    end

    sol = A \ b
    
    return sparse(A), x, b, sol
    end   

Ejercicio1_1 (generic function with 2 methods)

In [28]:
n = 20 # Número de nodos
inicio, final = 2, 6.5 # Extremos del intervalo
b = zeros(n)
b[1] = 0.008
b[end] = 0.003

A, x, b, sol = Ejercicio1_1(n, inicio, final, b);

## 1 Métodos iterativos estacionarios

### 1.1 Método de Jacobi

In [29]:
function Jacobi(A, b, iter = 20000)
    n = size(A)[1]
    x = [[] for i = 1:iter]
    x[1] = b
    x_bar = zeros(n)

    for k = 1:iter-1
        for i = 1:n
            x_bar[i] = 0
            for j = 1:n
                if j != i
                    x_bar[i] = x_bar[i] + A[i,j]*x[k][j]
                end
            end
            x_bar[i] = (b[i] - x_bar[i])/A[i,i]
        end
        #print(x_bar)
        x[k+1] = x_bar
    end
    return x[end]
end

Jacobi (generic function with 2 methods)

In [14]:
A, x, b, sol = Ejercicio1_1(n, inicio, final, b, false);

In [32]:
Jacobi(A,b) ≈ sol

true

### 1.2 Método de Gauss-Seidel

In [33]:
function GS(A, b, iter = 20000)
    n = size(A)[1]
    x = [[] for i = 1:iter]
    x[1] = b

    for k = 1:iter-1
        x[k+1] = zeros(n)
        for i = 1:n
            σ = 0
            for j = 1:i-1
                if j <= i
                    σ = σ + A[i,j]x[k+1][j]
                end
            end
            for j = i+1:n
                σ = σ + A[i,j]x[k][j]
            end
            x[k+1][i] = (b[i] - σ)/A[i,i]
        end
    end
    return x[end]
end 

GS (generic function with 2 methods)

In [34]:
GS(A, b) ≈ sol

true

### 1.3 Método de sobre-relajación sucesiva

### 1.4 Método de sobre-relajación sucesiva simétrica

## 2 Métodos iterativos no estacionarios

### 2.1  Método del Gradiente Conjugado (CG)

### 2.2  Residuo Mínimo (MINRES) y SYMMLQ

### 2.3  CG en las Ecuaciones Normales, CGNE y CGNR

### 2.4  Residuo Mínimo Generalizado (GMRES)

### 2.5 Gradiente BiConjugado (BiCG)

### 2.6  Cuasi-Mínimo Residuo (QMR)

### 2.7 Método del Gradiente Conjugado Cuadrático (CGS)

### 2.8 Gradiente BiConjugado Estabilizado (Bi-CGSTAB)

### 2.9 Iteración de Chebyshev